#### MVP: 
 * a streamlit dashboard and a tableau dashboard
 * filter by activities
 * **total time spent on activites by week**

#### ideas for this data: 
 * convert the map data to elevation data
 * something with the suffer score
   * suffer per second? suffer per minute? 

#### to do
 * convert start_date column to pandas timestamp 

In [1]:
import pandas as pd
pd.options.display.max_rows = 999

import time
import datetime as dt
import numpy as np
from pathlib import Path
import requests
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import streamlit as st
import time

pd.options.mode.chained_assignment = None

In [200]:
def get_strava_data(csv_file):
    with open('strava_tokens.json') as json_file:
        strava_tokens = json.load(json_file)

    if strava_tokens['expires_at'] < time.time():
        response = requests.post(
                            url = 'https://www.strava.com/oauth/token',
                            data = {
                                    'client_id': 88709,
                                    'client_secret': '54dab45b88619205ee1e47eb94d0f2d84762a4b2',
                                    'grant_type': 'refresh_token',
                                    'refresh_token': strava_tokens['refresh_token']
                                    }
                        ) 
        new_strava_tokens = response.json() 
        with open('strava_tokens.json', 'w') as outfile:
            json.dump(new_strava_tokens, outfile)
        strava_tokens = new_strava_tokens

    page = 1
    url = "https://www.strava.com/api/v3/activities"
    access_token = strava_tokens['access_token']

    activities = pd.read_csv(csv_file)

    while True:
        r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
        r = r.json()
        r_df = pd.DataFrame.from_dict(r, orient='columns')        
               
        # if there are no new values (i.e. activities) found in r_df, just break.  otherwise keep going
        if bool([v for v in list(activities['id']) if v not in list(r_df['id'])]):
            return activities  

        if (not r):   # if no results then exit loop
            break
            
        activities = pd.concat([r_df, activities])
        page += 1
    
    activities.drop_duplicates(subset=['id'], inplace=True)
    activities.sort_values(by='id', ascending=False, inplace=True)
    return activities

In [201]:
strava_activities_new = get_strava_data('strava_activities.csv')

In [227]:
# always remember to pass index=False to not add an Unnamed column on your export! 
strava_activities_raw.to_csv('strava_activities.csv', index=False)

In [2]:
strava_activities_raw = pd.read_csv('strava_activities.csv')

In [8]:
cols_to_drop = ['Unnamed: 0.1', 'athlete', 'resource_state', 'start_date_local', 'timezone', 
'utc_offset', 'trainer', 'commute', 'type', 'manual', 'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'display_hide_heartrate_option',
'upload_id_str', 'external_id', 'from_accepted_tag', 'total_photo_count', 'athlete.resource_state', 'map.id', 'map.summary_polyline', 'map.resource_state', 
'average_watts', 'kilojoules', 'device_watts', 'photo_count', 'heartrate_opt_out', 'upload_id', 'athlete.id', 'workout_type',
'has_heartrate', 'location_city', 'location_state', 'location_country', 'private', 'has_kudoed', 'Unnamed: 0']

strava_activities_clean = strava_activities_raw.drop(labels=cols_to_drop, axis=1)

In [15]:
strava_activities_clean.head(10)

,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,id,start_date,achievement_count,kudos_count,...,average_speed,max_speed,average_cadence,average_temp,average_heartrate,max_heartrate,elev_high,elev_low,pr_count,suffer_score
0,Afternoon Run,6.014199,3682,3685,314.0,Run,7395188303,2022-06-30T21:39:30Z,11,3,...,2.630,4.392,84.4,28.0,154.1,180.0,183.2,48.2,6,102.0
1,Afternoon Golf,6.437597,10586,14280,120.0,Golf,7390944431,2022-06-30T00:12:17Z,0,2,...,0.979,2.624,NaN,28.0,102.5,148.0,93.8,68.4,0,29.0
2,Lunch Run,4.054633,2165,2168,132.0,Run,7389543804,2022-06-29T19:24:38Z,3,1,...,3.016,4.302,87.0,27.0,150.8,167.0,181.4,49.0,0,47.0
3,Lunch Run,6.038169,3264,3304,61.0,Run,7384051727,2022-06-28T19:05:39Z,0,2,...,2.979,4.058,85.2,23.0,149.2,159.0,83.6,45.6,0,73.0
4,Morning Run,19.993964,12853,12931,362.0,Run,7373215314,2022-06-26T13:58:16Z,26,4,...,2.505,4.490,85.8,25.0,145.7,163.0,258.0,87.6,11,229.0
5,Lunch Run,6.079466,3128,3162,64.0,Run,7362386558,2022-06-24T18:38:54Z,0,3,...,3.130,4.310,87.0,27.0,148.9,167.0,85.4,47.0,0,63.0
6,Lunch Ride,7.954079,2925,2970,243.0,Ride,7357786241,2022-06-23T19:07:02Z,6,2,...,4.379,11.150,NaN,24.0,114.5,141.0,201.4,59.6,2,10.0
7,Morning Run,6.012522,3343,3384,76.0,Run,7352038723,2022-06-22T17:42:39Z,0,3,...,2.896,4.349,85.5,27.0,145.6,164.0,85.2,46.0,0,56.0
8,Evening Run,4.186223,2822,2847,280.0,Run,7348290559,2022-06-22T02:12:17Z,5,3,...,2.389,4.285,81.4,27.0,157.8,179.0,184.6,53.2,1,89.0
9,Afternoon Mountain Bike Ride,6.998049,4571,6088,460.0,Ride,7342540098,2022-06-21T00:57:44Z,5,3,...,2.465,10.048,NaN,16.0,139.5,171.0,269.8,70.6,2,64.0


In [34]:
# meters to miles
#strava_activities_clean['distance'] = strava_activities_clean.distance * .000621

# 1:01:22 ; moving_time is in seconds. let's just add another column for the datetime
#strava_activities_clean['suffer_score'][0] / (strava_activities_clean['moving_time'][0] / 60)
strava_activities_clean['SPM'] = strava_activities_clean.suffer_score / (strava_activities_clean.moving_time / 60)bb

In [42]:
strava_activities_clean.start_date[3]

'2022-06-28T19:05:39Z'

In [43]:
#strava_activities_clean.sort_values(by='SPM', ascending=False)
pd_dt_obj = pd.to_datetime(strava_activities_clean.start_date[3])

In [66]:
pd_dt_obj

Timestamp('2022-06-28 19:05:39+0000', tz='UTC')

In [65]:
pd_dt_obj + pd.Timedelta(3600, unit='sec')

Timestamp('2022-06-28 20:05:39+0000', tz='UTC')

In [57]:
regular_dt_obj

numpy.datetime64('2022-06-28T19:05:39.000000000')